<a href="https://colab.research.google.com/github/staller84/minhee/blob/master/feature_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 형태소 분석 테스트를 위한 패키지 설치

In [1]:
!git clone https://github.com/e9t/nsmc.git
!pip3 install konlpy

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 18.02 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.
     |████████████████████████████████| 19.4MB 120kB/s 
     |████████████████████████████████| 2.7MB 45.3MB/s 


In [2]:
!cat nsmc/ratings_train.txt | head -n 10

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1


###### 패키지 선언

In [3]:
# code
import json
import os
import nltk
import numpy as np

from pprint import pprint
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

###### 파일은 \t을 기준으로 구성되어 있음.


In [0]:
def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

###### 토큰화 방법 

In [0]:
def tokenize(doc, morph, case_check):
    if case_check == "Hannanum":
        return ['/'.join(t) for t in morph.pos(doc)]
    elif case_check == "Kkma":
        return ['/'.join(t) for t in morph.pos(doc)]  
    elif case_check == "Komoran":
        return ['/'.join(t) for t in morph.pos(doc)]    
    elif case_check == "Okt":
        return ['/'.join(t) for t in morph.pos(doc)]
    elif case_check == "Syl":
        return [t for t in doc]
    else:
        data = doc.split(" ")
        return data

###### 토큰화 시켜서 데이터 재생성

In [0]:
def gen_data(morph, case_check):
    train_data = read_data('./nsmc/ratings_train.txt')
    test_data = read_data('./nsmc/ratings_test.txt')
    train_data = train_data[:10000]
    test_data = test_data[:1000]
    print("학습데이터 형태 및 수")
    print(train_data[:1])
    print(len(train_data))
    print()
    print("테스트데이터 형태 및 수")
    print(len(test_data))
    print(test_data[:1])
    print()
    
    print("gen new data")
    train_docs = [(tokenize(row[1], morph, case_check), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1], morph, case_check), row[2]) for row in test_data]
    
    return train_docs, test_docs

###### 토큰 갯수 세기 함수

In [0]:
def count_token(train_docs):
    tokens = [t for d in train_docs for t in d[0]]
    print("토큰수를 세기위한 샘플 토큰")
    print(tokens[0])
    print()
    text = nltk.Text(tokens, name='NMSC')
    
    # 전체 토큰의 개수
    print("전체 토큰의 개수")
    print(len(text.tokens))
    print()
    # 중복을 제외한 토큰의 개수
    print("중복을 제외한 토큰의 개수")
    print(len(set(text.tokens)))
    print()
    # 출현 빈도가 높은 상위 토큰 10개
    print("출현 빈도가 높은 상위 토큰 10개")
    print(text.vocab().most_common(10))
    print()
    return text

###### 모델선언
###### CountVectorization을 사용하여 데이터를 벡터화
###### 문서집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 Bag Of Words 인코딩 방식으로 벡터를 표현

In [0]:
def term_frequency(doc, selected_words):
    return [doc.count(word) for word in selected_words]


def set_input(text, train_docs, test_docs):
    selected_words = [f[0] for f in text.vocab().most_common(1500)]
    train_x = [term_frequency(d, selected_words) for d, _ in train_docs]
    test_x = [term_frequency(d, selected_words) for d, _ in test_docs]
    train_y = [c for _, c in train_docs]
    test_y = [c for _, c in test_docs]
    
    # data 형 변환
    x_train = np.asarray(train_x).astype('float32')
    x_test = np.asarray(test_x).astype('float32')
    y_train = np.asarray(train_y).astype('float32')
    y_test = np.asarray(test_y).astype('float32')
    return selected_words, x_train, x_test, y_train, y_test


def set_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(1500,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

###### 학습

In [0]:
def train_model(model, x_train, y_train):
    print("\n training part")
    model.fit(x_train, y_train, epochs=5, batch_size=32)
    return model

###### 평가

In [0]:
def eval_model(model, x_test, y_test):
    print("\n evaluation part")
    results = model.evaluate(x_test, y_test)
    print(results)

###### 메인함수

In [14]:
if __name__ == '__main__':
    # 1 한나눔 형태소 = 72.1%
#     case_check = "Hannanum"
#     morph = Hannanum()

    # 2 꼬꼬마 형태소 = 80.1%
#     case_check="Kkma"
#     morph = Kkma()
    
    # 3 코모란 형태소 = 76.9%
#     case_check="Komoran"
#     morph = Komoran()

    # 4 Okt 형태소 = 77.4%
    # case_check="Okt"
    # morph = Okt()

    # 5 단순 어절 = 66.7%
    case_check = None
    morph = None
    
    # 6 단순 음절 = 76.1%
#     case_check = "Syl"
#     morph = None
    

    train_docs, test_docs = gen_data(morph, case_check)
    # 학습데이터 샘플
    print(train_docs[0])
    
    text = count_token(train_docs)
    selected_words, x_train, x_test, y_train, y_test = set_input(text, train_docs, test_docs)
    
    model = set_model()

    model = train_model(model, x_train, y_train)
    eval_model(model, x_test, y_test)

학습데이터 형태 및 수
[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']]
10000

테스트데이터 형태 및 수
1000
[['6270596', '굳 ㅋ', '1']]

gen new data
(['아', '더빙..', '진짜', '짜증나네요', '목소리'], '0')
토큰수를 세기위한 샘플 토큰
아

전체 토큰의 개수
77210

중복을 제외한 토큰의 개수
40299

출현 빈도가 높은 상위 토큰 10개
[('영화', 737), ('정말', 534), ('너무', 508), ('진짜', 391), ('이', 330), ('영화.', 262), ('그냥', 235), ('이런', 233), ('왜', 223), ('더', 209)]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

 training part
Train on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 1s 125us/sample - loss: 0.6157 - acc: 0.6488
Epoch 2/5
10000/10000 [==============================] - 1s 76us/sample - loss: 0.4775 - acc: 0.7411
Epoch 3/5
10000/10000 [==============================] - 1s 77us/sample - loss: 0.4187 - acc: 0.7729
Epoch 4/5
10000/10000 [==============================] - 1s 79us/sample - loss: 0.3713 - acc: 0.7988
Epoch 5/5